<a href="https://colab.research.google.com/github/poenzel/sofina/blob/main/Sofina_Dataset_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extraction de datasets (csv) à partir de pdf
Ce notebook emploie des méthodes de détection de tables afin d'extraire les informations. Passer par des modèles de ML entrainés est dû au grand nombre de formats différents employant des cellules mergées, ainsi que la présence de watermark dans ces pdfs qui recouvrent les tables.

## Ordre des opérations :    
### 1. Charger le(s) pdf(s)
### 2. Transformer chaque page du(des) pdf(s) en image
### 3. Détecter sur l'image les tables
### 4. Identifier les cellules ainsi que leur contenu
### 5. S'assurer que les cellules soient alignés afin d'avoir un tableau cohérent
### 6. Merge le tout

## Après l'extraction :
Une fois les données à disposition, des étapes de cleaning supplémentaires seront encore nécessaires. Cependant, le contenu des cellules sera correct. Il faudra juste éliminer les termes parasites restants (par exemple, '.com' qui apparait à cause de la watermark) et adapter le format si nécessaire.

Certains .csv ne vont contenir que du texte sur une colonne => on peut les convertir en texte à étudier si besoin. L'idée ici c'est qu'on extrait un maximum d'information.

Par endroits, certaines colonnes sont "coupées"; cela vaudrait la peine de voir si d'autres modèles permettraient de mieux identifier les zones tabulaires, ou si on est capable de fine-tune ce modèle-ci pour nos cas.

Les données ne sont pas parfaites pour l'instant, mais l'objectif c'est d'avoir un délivrable pour une "démonstration". On pourra polir tout le reste par la suite, quand un budget sera débloqué ou même en continuant d'avancer sur le end product, si tout va bien.

# Importer les packages & autres dépendances

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pdf2image
!apt-get install poppler-utils

In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

In [ ]:
!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf==3.20.0
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

In [6]:
import os
from pdf2image import convert_from_path

In [7]:
import tensorflow as tf
import numpy as np
import cv2
import layoutparser as lp
import pandas as pd
from paddleocr import PaddleOCR, draw_ocr

/usr/local/lib/python3.10/dist-packages/paddle/base/framework.py:688: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


## Transformer les pdfs en série d'images
Afin de les traiter convenablement, il faut créer un folder spécifique pour chaque pdf.

In [ ]:
# Create the subfolders that will contain each image of each page for each pdf file
main_path = '/content/drive/MyDrive/Sofina/Dataset1'

for folder in os.listdir(main_path) :
  # Define path for each company folder
  folder_path = os.path.join(main_path, folder)

  for file in os.listdir(folder_path) :
    # If it's not a pdf file, skip
    if os.path.splitext(file)[1] != '.pdf' :
      continue

    # Define path for each pdf file
    file_path = os.path.join(folder_path, file)
    # Get the name of the pdf file, so that we know where to store the pages
    pdf_name = os.path.splitext(file_path)[0]

    pages_folder = os.path.join(folder_path,pdf_name)
    print('Creating folder :', pages_folder)
    if os.path.isdir(pages_folder) :
      print('The folder already exists.')
    else :
      os.mkdir(pages_folder)
      print('New folder has been created :', pages_folder)

    # Convert each page into a jpg file and store it in the newly created folder
    images = convert_from_path(file_path)

    for i in range(len(images)):
      page_prefix_path = os.path.join(pages_folder,'page')
      images[i].save(page_prefix_path+str(i)+'.jpg','JPEG')

# Extraire les données tabulaires des images

In [22]:
def page_to_dataset(folder_path, page_path) :
    # Get the path of the page
    page = os.path.splitext(os.path.basename(page_path))[0]

    # Create a specific folder to store temporary images & output datasets for that page
    page_dataset_path = os.path.join(folder_path, page)
    print('Creating folder :',page_dataset_path)
    if os.path.isdir(page_dataset_path) :
      print('The folder already exists.')
    else :
      os.mkdir(page_dataset_path)
      print('New folder has been created :', page_dataset_path)

    # Initiate the model for table/text detection
    image = cv2.imread(page_path)
    image = image[..., ::-1]

    model = lp.PaddleDetectionLayoutModel(config_path= "lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                        threshold = 0.5,
                                        label_map= {0: "Text", 1: "Title", 2: "List", 3: "Table", 4:"Figure"},
                                        enforce_cpu = True,
                                        enable_mkldnn= True)
    layout = model.detect(image)

    # Initiate the Optical Character Recognition tool
    ocr = PaddleOCR(lang='en', use_gpu = False)

    box_nb = 0
    #Let's iterate through each detected zones and compute the resulting dataset (even if it's text)
    for layout_block in layout :
        # Note : this condition should actually skip each block except Tables, but due to special formats (and a gigantic
        # watermark that covers everything)
        # if layout_block.type  == 'Title' :
        #     box_nb += 1
        #     continue

        # Get the coordinates of the frame of the detected text entity
        x_1 = int(layout_block.block.x_1)
        y_1 = int(layout_block.block.y_1)
        x_2 = int(layout_block.block.x_2)
        y_2 = int(layout_block.block.y_2)

        # Define a temporary image on which identifying boxes around values
        temp_img = page +'_temp1_'+str(box_nb) + '.jpg'
        temp_img_path = os.path.join(page_dataset_path,temp_img)
        im2 = cv2.imread(page_path)
        cv2.imwrite(temp_img_path, im2[y_1:y_2,x_1:x_2])


        image_cv = cv2.imread(temp_img_path)
        image_height = image_cv.shape[0]
        image_width = image_cv.shape[1]

        # # Initiate the Optical Character Recognition tool
        # ocr = PaddleOCR(lang='en', use_gpu = False)
        output = ocr.ocr(temp_img_path)

        if output[0] is None :
          continue

        boxes = [line[0] for line in output[0]]
        texts = [line[1][0] for line in output[0]]
        probabilities = [line[1][1] for line in output[0]]

        # Define another temporary image to get & check if values & cells are correctly identified
        temp_img_2 = page + '_temp2_' + str(box_nb) + '.jpg'
        temp_img_2_path = os.path.join(page_dataset_path,temp_img_2)
        image_boxes = image_cv.copy()
        for box,text in zip(boxes,texts):
            cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
            cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)
            cv2.imwrite(temp_img_2_path, image_boxes)

        # Let's rebuild the interpreted dataset from what has been detected
        im = image_cv.copy()
        horiz_boxes = []
        vert_boxes = []

        for box in boxes:
          x_h, x_v = 0,int(box[0][0])
          y_h, y_v = int(box[0][1]),0

          # Difference between the min & max x-coordinate of the box
          width_h,width_v = image_width, int(box[2][0]-box[0][0])
          # Same on the y axis
          height_h,height_v = int(box[2][1]-box[0][1]),image_height

          horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
          vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

          cv2.rectangle(im,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
          cv2.rectangle(im,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

        temp_img_3 = page + '_temp3_' + str(box_nb) + '.jpg'
        temp_img_3_path = os.path.join(page_dataset_path,temp_img_3)
        cv2.imwrite(temp_img_3_path,im)

        # Keep only the boxes with the best scores (the least IOU)
        # We can play with the iou_threshold to get + or - boxes
        horiz_out = tf.image.non_max_suppression(
            horiz_boxes,
            probabilities,
            max_output_size = 500,
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )


        temp_img_4 = page+ '_finalBox_' + str(box_nb) + '.jpg'
        temp_img_4_path = os.path.join(page_dataset_path, temp_img_4)

        horiz_lines = np.sort(np.array(horiz_out))
        im_nms = image_cv.copy()
        for val in horiz_lines:
            cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)
        cv2.imwrite(temp_img_4_path,im_nms)

        # Keep only the boxes with the best scores again (vertical boxes in this case)
        # Again, we can play with the iou_threshold if needed
        vert_out = tf.image.non_max_suppression(
            vert_boxes,
            probabilities,
            max_output_size = 100, # we could get the maximum table size manually
            iou_threshold=0.1,
            score_threshold=float('-inf'),
            name=None
        )
        vert_lines = np.sort(np.array(vert_out))

        for val in vert_lines:
            cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

        cv2.imwrite(temp_img_4_path,im_nms)

        # Transform the resulting image into a table
        out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
        unordered_boxes = []

        for i in vert_lines:
            unordered_boxes.append(vert_boxes[i][0])
        ordered_boxes = np.argsort(unordered_boxes)

        def intersection(box_1, box_2):
            return [box_2[0], box_1[1],box_2[2], box_1[3]]

        def iou(box_1, box_2):

            x_1 = max(box_1[0], box_2[0])
            y_1 = max(box_1[1], box_2[1])
            x_2 = min(box_1[2], box_2[2])
            y_2 = min(box_1[3], box_2[3])

            inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
            if inter == 0:
                return 0

            box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
            box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

            return inter / float(box_1_area + box_2_area - inter)

        for i in range(len(horiz_lines)):
            for j in range(len(vert_lines)):
                resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

                for b in range(len(boxes)):
                    the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
                    if(iou(resultant,the_box)>0.1):
                        out_array[i][j] = texts[b]

        out_array=np.array(out_array)

        output_dataset = page + '_dataset_' + str(box_nb) + '.csv'
        output_dataset_path = os.path.join(page_dataset_path, output_dataset)
        pd.DataFrame(out_array).to_csv(output_dataset_path)

        box_nb +=1

    for filename in os.listdir(page_dataset_path) :
      if os.path.splitext(filename)[1] != '.jpg' or 'finalBox' in filename :
        continue
      else :
        os.remove(os.path.join(page_dataset_path,filename))

In [ ]:
### Too much RAM required ###

# from concurrent.futures import ThreadPoolExecutor

# # Iterate over each company folder
# for folder in os.listdir(main_path):
#     folder_path = os.path.join(main_path, folder)

#     # Access each subfolder within the company folder
#     for subfolder in os.listdir(folder_path):
#         subfolder_path = os.path.join(folder_path, subfolder)

#         # Check if the subfolder is a directory
#         if os.path.isdir(subfolder_path):
#             # Use a ThreadPoolExecutor to parallelize image processing
#             with ThreadPoolExecutor() as executor:
#                 # Iterate over each image in the subfolder
#                 for page_img in os.listdir(subfolder_path):
#                   if os.path.splitext(page_img)[1] != '.jpg':
#                     continue
#                   # Submit the image processing task to the executor
#                   page_path = os.path.join(subfolder_path, page_img)
#                   executor.submit(page_to_dataset, subfolder_path, page_path)


In [ ]:
### THIS TAKES A LONG TIME if run on all folders (3-4h) --> Better run a folder at a time. ###
chosen_folder = ['THOMA BRAVO']

for folder in os.listdir(main_path) : # change "in os.listdir(main_path)" to "in chosen_folder" to execute on a single company
  folder_path = os.path.join(main_path, folder)
  # Access to each company folder
  for subfolder in os.listdir(folder_path) :
    subfolder_path = os.path.join(folder_path, subfolder)
    # If we access a folder we created before
    if os.path.isdir(subfolder_path) :
      # Access to each image and run the function defined above for each image
      for page_img in os.listdir(subfolder_path) :
        if os.path.splitext(page_img)[1] != '.jpg' :
          continue
        page_path = os.path.join(subfolder_path,page_img)
        page_to_dataset(subfolder_path, page_path)

## En dessous :
Tentatives d'exploiter les coordonnées obtenues. Un des problèmes vient de la watermark...

In [54]:
!pip install tabula-py

ERROR: Operation cancelled by user


In [55]:
!pip install img2table

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.4/91.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 11.0 MB/s eta 0:00:00


In [ ]:
# import tabula
from img2table.document import Image

path = '/content/drive/MyDrive/Sofina/Dataset1/ANDREESSEN & HOROWITZ/pages/ext_im.jpg'

img = Image(src=path)

# Table identification
img_tables = img.extract_tables()

# Result of table identification
img_tables
# custom_area = (208.52716064453125-104, 284.2144775390625-142, 1995.473876953125+104, 1469.7366943359375+142)

#TODO : Split la merged cell pcq même avec une custom area ça passe pas

# dfs = tabula.read_pdf(path, stream = True,
#                       # area= custom_area,
#                       pages= '6')

# dfs[0].head(20)